# Implementation

In [1]:
# imports
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os
import cv2

import keras
import tensorflow as tf

from keras.applications import InceptionV3

In [2]:
# file paths
path = os.getcwd()
path = path[:-4]
data_dir = path + "/data/lfw-deepfunneled"

In [3]:
# get list of LFW names to use: intersection of names in 'people.csv' with names in 'os.listdir(data_dir)'
# specify min # of images each LFW subject should have to be considered in experiment: N
# N yields k classes (LFW subjects)
people = pd.read_csv(path + "/data/people.csv")
names = [item for item in people["name"].tolist() if item in os.listdir(data_dir)] # intersection of names
people = people[people["name"].isin(names)] # filter people DataFrame to intersection

N = 6
people = people[people.images > N]

k = len(people)
print(f"k = {k} classes")

k = 256 classes


In [4]:
# model architecture: pre-trained InceptionV3 transfer learning to k subjects of LFW dataset
inceptionv3_transfer = InceptionV3(include_top=False, classes=k, pooling='avg', weights='imagenet')
model = keras.Sequential(layers=(inceptionv3_transfer, keras.layers.Dense(k, activation='softmax')))
model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=1e-4), metrics=['accuracy'])

## Data Preprocessing $\rightarrow$ $X_{tr}, y_{tr}, X_{val}, y_{val}$

In [5]:
# restructure DataFrame
people = people.rename(columns={'images': 'num_images'})
people = people.set_index('name')

# process 250x250 image data as 299x299 image elements under respective person
image_col = []
for person in people.iterrows():
    person = person[1]
    image_list = []
    for i in range(int(person["num_images"])):
        image_path = data_dir + '/' + person.name + '/' + person.name + '_' +  "{:04d}".format(i+1) + '.jpg'
        image = cv2.imread(image_path)
        image = cv2.resize(image, (299, 299))
        image_list.append(image)
    image_col.append(image_list)
people["images"] = image_col

# num_images == len(images) for each person
assert ([len(x) for x in people["images"]] == people["num_images"].values).all()

# construct dataset DataFrame (precursor to X, y) with each row sample as the individual images
dataset = [] # element: (image, person, train or test)
for person in people.iterrows():
    person = person[1]
    image_list = person["images"]
    num_images = len(image_list)
    num_test_images = int(np.floor(num_images * 0.2))
    test_i = np.random.choice(num_images, num_test_images)
    for i in range(num_images):
        image = image_list[i]
        element = [image, person.name, "test" if i in test_i else "train"]
        dataset.append(element)
dataset = pd.DataFrame(dataset, columns=["image", "person", "split"])

# one-hot encode 'person' (will be used as y)
dataset = pd.get_dummies(dataset, columns=['person'])

In [6]:
dataset.head()

,image,split,person_Abdullah_Gul,person_Adrien_Brody,person_Al_Gore,person_Al_Sharpton,person_Alejandro_Toledo,person_Ali_Naimi,person_Alvaro_Uribe,person_Amelia_Vega,...,person_Walter_Mondale,person_Wen_Jiabao,person_William_Donaldson,person_William_Ford_Jr,person_Winona_Ryder,person_Yao_Ming,person_Yashwant_Sinha,person_Yasser_Arafat,person_Yoriko_Kawaguchi,person_Zhu_Rongji
0,"[[[0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], ...",train,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,"[[[0, 1, 1], [0, 1, 1], [0, 1, 1], [0, 1, 1], ...",train,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,"[[[0, 0, 0], [0, 0, 0], [1, 0, 0], [2, 0, 0], ...",train,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...",train,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,"[[[129, 163, 176], [129, 163, 176], [129, 163,...",test,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [7]:
# specify X, y
X_train = dataset[dataset["split"] == "train"]["image"].tolist()
X_train = np.asarray(X_train).astype('float32')
X_test = dataset[dataset["split"] == "test"]["image"].tolist()
X_test = np.asarray(X_test).astype('float32')

y_train = dataset[dataset["split"] == "train"].drop(['image', 'split'], axis=1)
y_test = dataset[dataset["split"] == "test"].drop(['image', 'split'], axis=1)

## Model Training

In [8]:
model.fit(X_train, y_train, shuffle=True, epochs=10)

Epoch 1/10
133/133 ━━━━━━━━━━━━━━━━━━━━ 1069s 8s/step - accuracy: 0.1661 - loss: 4.7156
Epoch 2/10
133/133 ━━━━━━━━━━━━━━━━━━━━ 1075s 8s/step - accuracy: 0.5840 - loss: 2.1550
Epoch 3/10
133/133 ━━━━━━━━━━━━━━━━━━━━ 1076s 8s/step - accuracy: 0.8695 - loss: 0.8649
Epoch 4/10
133/133 ━━━━━━━━━━━━━━━━━━━━ 1093s 8s/step - accuracy: 0.9826 - loss: 0.2887
Epoch 5/10
133/133 ━━━━━━━━━━━━━━━━━━━━ 1072s 8s/step - accuracy: 0.9986 - loss: 0.0872
Epoch 6/10
133/133 ━━━━━━━━━━━━━━━━━━━━ 1101s 8s/step - accuracy: 1.0000 - loss: 0.0351
Epoch 7/10
133/133 ━━━━━━━━━━━━━━━━━━━━ 1087s 8s/step - accuracy: 1.0000 - loss: 0.0218
Epoch 8/10
133/133 ━━━━━━━━━━━━━━━━━━━━ 1079s 8s/step - accuracy: 1.0000 - loss: 0.0139
Epoch 9/10
133/133 ━━━━━━━━━━━━━━━━━━━━ 4814s 36s/step - accuracy: 1.0000 - loss: 0.0105
Epoch 10/10
133/133 ━━━━━━━━━━━━━━━━━━━━ 7228s 55s/step - accuracy: 1.0000 - loss: 0.0081


## Model Evaluation

In [9]:
model.evaluate(X_test, y_test)

27/27 ━━━━━━━━━━━━━━━━━━━━ 425s 16s/step - accuracy: 0.8692 - loss: 0.5198


[0.4817023277282715, 0.8824228048324585]

In [11]:
# save model
model.save('model.keras')